<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/Simulationofanoilstabilizationprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simulation of an oil stabilization process
In this notebook we will establish code for using NeqSim for simulation of an oil stabilization process

In [1]:
# Setting up the Colab page to use neqsim
%%capture
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *

# Process flow diagram

In [2]:
inputdata = {
  'feedFlowRateWell': 16000,
  'wellPressure': 180.0,
  'wellTemperature':100.0,
  'topsidePressure': 90.0,
  'topsideTemperature': 5.0,
  'chokePressure': 80.0,



  'feedTemperature': 43.8, 
  'firstStageTemperature': 78.5,
  'firstStagePressure': 70.0, 
  'secondStagePressure': 24.6, 
  'temperatureOilHeater' : 55.9,
  'thirdStagePressure': 1.9,
  'secondStageRecompPressure': 6.6,
  'thirdStageRecompPressure': 12.7,  
  'fourthStageRecompPressure': 24.7,
  'exportCompressorPressure': 123.7,
  'firstStageSuctionCoolerTemperature': 25.3, 
  'secondStageSuctionCoolerTemperature': 24.5, 
   'thirdStageSuctionCoolerTemperature': 30.0, 
   'fourthStageSuctionCoolerTemperature': 25.0, 
  'firstStageCompressorOutTemperature': 125.3, 
  'secondStageCompressorOutTemperature': 90.0,
  'thirdStageCompressorOutTemperature': 93.7, 
  'fourthStageCompressorOutTemperature': 132.9, 
  'exportCompressorOutTemperature': 90.9, 
  'suctionCoolerOutTemperature': 25.0, 
  'exportCoolerTemp': 35.0,
  'exportGasFlowRate': 10000.0
}

In [3]:
from neqsim.thermo import fluid, printFrame
from neqsim.process import compressor, cooler, separator3phase, getProcess, clearProcess, mixer, heater, stream, pump, separator, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber,distillationColumn, waterStripperColumn, recycle2, setpoint, calculator

clearProcess()

wellFluid = fluid('pr')
wellFluid.addComponent("nitrogen", 0.08)
wellFluid.addComponent("CO2", 3.56)
wellFluid.addComponent("methane", 87.36)
wellFluid.addComponent("ethane", 4.02)
wellFluid.addComponent("propane", 1.54)
wellFluid.addComponent("i-butane", 0.2)
wellFluid.addComponent("n-butane", 0.42)
wellFluid.addComponent("i-pentane", 0.15)
wellFluid.addComponent("n-pentane", 0.20)
		
wellFluid.addTBPfraction("C6", 0.24, 84.99/1000.0, 695.0/1000.0)
wellFluid.addTBPfraction("C7", 0.34, 97.87/1000.0, 718.0/1000.0)
wellFluid.addTBPfraction("C8", 0.33, 111.54/1000.0, 729.0/1000.0)
wellFluid.addTBPfraction("C9", 0.19, 126.1/1000.0, 749.0/1000.0)
wellFluid.addTBPfraction("C10", 0.15, 140.14/1000.0, 760.0/1000.0)
wellFluid.addTBPfraction("C11", 0.69, 175.0/1000.0, 830.0/1000.0)
wellFluid.addTBPfraction("C12", 0.5, 280.0/1000.0, 914.0/1000.0)
wellFluid.addTBPfraction("C13", 0.103, 560.0/1000.0, 980.0/1000.0)

wellFluid.setMixingRule(2)
wellFluid.init(0)

wellFluid.setMolarComposition([0.08, 3.56, 87.36, 4.02, 1.54, 0.2, 0.42, 0.15, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.24, 0.34, 0.33, 0.19, 0.15, 0.69, 0.5, 0.03])

wellStream = stream(wellFluid)
wellStream.setName("dry well stream")
wellStream.setFlowRate(inputdata['feedFlowRateWell'], "kg/hr")
wellStream.setTemperature(inputdata['wellTemperature'], "C")
wellStream.setPressure(inputdata['wellPressure'], "bara")

saturatedFeedGas = saturator(wellStream)
saturatedFeedGas.setName("water saturator")

waterSaturatedFeedGas = stream(saturatedFeedGas.getOutStream())
waterSaturatedFeedGas.setName("water saturated feed gas")

feedTPsetter = heater(waterSaturatedFeedGas)
feedTPsetter.setName('inletTP')
feedTPsetter.setOutPressure(inputdata['topsidePressure'], "bara")
feedTPsetter.setOutTemperature(inputdata['topsideTemperature'], "C")

chokeValve = valve(feedTPsetter.getOutStream())
chokeValve.setOutletPressure(inputdata['chokePressure'], 'bara')

feedToOffshoreProcess = stream(chokeValve.getOutStream())
feedToOffshoreProcess.setName("feed to offshore")

firstStageSeparator = separator3phase(feedToOffshoreProcess)
firstStageSeparator.setName("1st stage separator")

oilHeaterFromFirstStage = heater(firstStageSeparator.getOilOutStream())
oilHeaterFromFirstStage.setName("oil heater second stage")
oilHeaterFromFirstStage.setOutTemperature(273.15+inputdata['temperatureOilHeater'])

oilThrotValve = valve(oilHeaterFromFirstStage.getOutStream())
oilThrotValve.setName("valve oil from first stage")
oilThrotValve.setOutletPressure(inputdata['secondStagePressure'])

secondStageSeparator = separator3phase(oilThrotValve.getOutStream())
secondStageSeparator.setName("2nd stage separator")

oilThrotValve2 = valve(secondStageSeparator.getOilOutStream())
oilThrotValve2.setName("valve oil from second stage")
oilThrotValve2.setOutletPressure(inputdata['thirdStagePressure'])

thirdStageSeparator = separator3phase(oilThrotValve2.getOutStream())
thirdStageSeparator.setName("3rd stage separator")

oilThirdStageToSep =  wellStream.clone()
oilThirdStageToSep.setName("resyc oil")
thirdStageSeparator.addStream(oilThirdStageToSep)

stableOil = stream(thirdStageSeparator.getOilOutStream())
stableOil.setName("stable oil")

stableOilPump = pump(stableOil,15.0,"stable oil pump")

firstStageCooler = cooler(thirdStageSeparator.getGasOutStream())
firstStageCooler.setName("1st stage cooler")
firstStageCooler.setOutTemperature(273.15+inputdata['firstStageSuctionCoolerTemperature'])

firstStageScrubber = separator(firstStageCooler.getOutStream())
firstStageScrubber.setName("1st stage scrubber")

firstStageCompressor = compressor(firstStageScrubber.getGasOutStream())
firstStageCompressor.setName("1st stage compressor")
firstStageCompressor.setOutletPressure(inputdata['secondStageRecompPressure'])

secondStageCooler = cooler(firstStageCompressor.getOutStream())
secondStageCooler.setName("2nd stage cooler")
secondStageCooler.setOutTemperature(273.15+inputdata['secondStageSuctionCoolerTemperature'])

secondStageScrubber = separator(secondStageCooler.getOutStream())
secondStageScrubber.setName("2nd stage scrubber")

secondStageCompressor = compressor(secondStageScrubber.getGasOutStream())
secondStageCompressor.setName("2nd stage compressor")
secondStageCompressor.setOutletPressure(inputdata['chokePressure'])

richGasMixer = mixer("fourth Stage mixer")
richGasMixer.addStream(secondStageCompressor.getOutStream())
richGasMixer.addStream(firstStageSeparator.getGasOutStream())

dewPointControlCooler = cooler(richGasMixer.getOutStream())
dewPointControlCooler.setName("dew point cooler")
dewPointControlCooler.setOutTemperature(273.15+inputdata['fourthStageSuctionCoolerTemperature'])

dewPointScrubber = separator(dewPointControlCooler.getOutStream())
dewPointScrubber.setName("dew point scrubber")

lpLiqmixer = mixer("LP liq gas mixer");
lpLiqmixer.addStream(firstStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(secondStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(dewPointScrubber.getLiquidOutStream());

lpResycle = recycle2("LP liq resycle")
lpResycle.addStream(lpLiqmixer.getOutStream())
lpResycle.setOutletStream(oilThirdStageToSep)

oilprocess = getProcess()


# Start simulation

In [4]:
thread = oilprocess.runAsThread()

# Check if simulation has converged/stopped

True if still running.
False if finished.

In [16]:
thread.isAlive() 

True

# Read results from simulation

In [15]:
powerComp1 = oilprocess.getUnit("1st stage compressor").getPower()/1.0e3
print('recomp power [kW] ', powerComp1)


recomp power [kW]  1684.1360502587588
